In [ ]:
# mamba create -n pyvista -c conda-forge pyvista trame python=3.10 ipykernel ipywidgets
# mamba install -c conda-forge xarray vtk
import numpy as np
import pyvista as pv
import xarray as xr
import gc
import sys
import os

In [ ]:
# # pv.start_xvfb()
# # pv.set_jupyter_backend('trame')
# #Test Animation
# sphere = pv.Sphere()

# pl = pv.Plotter()
# actor = pl.add_mesh(sphere)
# # pl.screenshot('hey.png')

In [ ]:
# # Create meshgrid.

# grid = pv.RectilinearGrid(x,y,z)
# grid

# grid.point_data["clouds"] = cloud_array.flatten(order="F")
# grid.plot(show_edges=True)

In [ ]:
def plotdata(fname,sdir):
    pv.start_xvfb()
    # Open file and get x,y,z coordinates.
    data = xr.open_dataset(fname)
    x=data['xh'].values
    y=data['yh'].values
    z=data['zh'].values
    qc = data['qc'].isel(time=1200)
    qi = data['qi'].isel(time=1200)
    b = data['buoyancy'].isel(time=1200)
    w = data['w'].isel(time=1200)
    cloud_array=findclouds(qc,qi)

    # Initialize pyvista grid and add data.
    grid = pv.RectilinearGrid(x,y,z)

    # Now find clouds and create isosurface.
    # grid.cell_data["clouds"] = findclouds(qc,qi).flatten(order="F") 
    grid.point_data["clouds"] = cloud_array.flatten(order="F") #*
    isosurf = grid.contour([1], scalars="clouds")

    #Static image.
    p = pv.Plotter(notebook=False,off_screen=True, 
                 window_size=[1920*2, 1080*2])
                   #, multi_samples=16) #*

    ##
    ## Temporary hack to enable depth peeling. This helps the isosurface rendering look much
    ## better when stacked in front of other semitransparent isosurfaces. This will likely be
    ## added as an option in PyVista soon, but in the meantime, can manually be set through
    ## the plotter class.
    ##

    # 1. Use a render window with alpha bits (as initial value is 0 (false)):
    # allows for proper rendering of transparent or semi-transparent objects.
    p.ren_win.SetAlphaBitPlanes(True)

    # 2. Force to not pick a framebuffer with a multisample buffer
    # (as initial value is 4):
    #  helps reduce jagged edges in rendered images
    p.ren_win.SetMultiSamples(0)

    # 3. Choose to use depth peeling (if supported) (initial value is 0 (false)):
    # handle transparency by rendering transparent surfaces in multiple passes
    p.renderer.SetUseDepthPeeling(True)

    # 4. Set depth peeling parameters
    #  - Set the maximum number of rendering passes (initial value is 4):
    p.renderer.SetMaximumNumberOfPeels(4)
    # determines how many layers of transparent surfaces will be peeled away during rendering
    
    #  - Set the occlusion ratio (initial value is 0.0, exact image):
    # determines how much of the scene needs to be
    p.renderer.SetOcclusionRatio(0.0)

    #Static window.
    p.add_mesh(isosurf,scalars="clouds", cmap="Reds", opacity=0.5, clim=[0,1e-6],show_scalar_bar=False)

    #Interactive window. Requires Uniform grid.
    """
    grid.set_active_scalar("qc")
    p.add_volume(grid,scalars="qc",clim=[1e-7,3e-7],cmap="Greys_r", opacity = "sigmoid_6", mapper="gpu")

    """
    p.camera_position = [(-150, -400, 450), # Eye position
    (10, 32, 2),  # Point of focus
    (0.5, 5, 1.2)] # Camera orentiation
    p.camera.zoom(1)
    p.camera.elevation = 30

    p.show_grid()
    p.show_bounds(show_xaxis=True,show_yaxis=True,show_zaxis=True,show_zlabels=True)
    
    os.makedirs(sdir + '/movie', exist_ok=True)
    
    ##
    ## Use for Screenshots
    ##
    
    savename = (sdir + '/movie/' + 'cloud_movie-'+str(1)+'.png')
    #  p.show(auto_close=False)
    #  p.write_frame()
    p.screenshot(savename)
    
    ##
    ## Use for Animation
    ##
    
    #p.show(auto_close=False)
    #p.write_frame()
    #p.open_movie(sdir + 'cloud_movie.mp4', framerate=60)
    
    
    pv.close_all()

#*****************************************************************************
plotdata('/mnt/lustre/koa/koastore/torri_group/air/cm1r20.3/run/cm1out_test3.nc',
        '/mnt/lustre/koa/koastore/torri_group/air')


In [ ]:
def findclouds(qc,qi):

    # Set threshold.
    thresh = 10e-5
    cloud_thresh= np.where(qc > thresh) or np.where(qi > thresh)

    # Initialize.
    cloud = np.zeros_like(qc) 

    # Find all qualifying updrafts. 
    cloud[cloud_thresh] = 1
    
    return cloud

def findupdrafts(qc,qi,b):

    # Set threshold.
    thresh = 10e-5
    updraft_thresh=np.where( ( (qc > thresh) | (qi > thresh) ) & (b>0))

    # Initialize.
    updraft = np.zeros_like(qc) 

    # Find all qualifying updrafts. 
    updraft[updraft_thresh] = 1
    
    return updraft

def findcores(qc,qi,w):

    # Set threshold.
    thresh = 10e-5
    core_thresh=np.where( ( (qc > thresh) | (qi > thresh) ) & (w>0.5))

    # Initialize.
    core = np.zeros_like(qc) 

    # Find all qualifying updrafts. 
    core[core_thresh] = 1
    
    return core

In [ ]:
# data_array = xr.DataArray(cloud_array, dims=('dim1', 'dim2', 'dim3'), name='var')

# # Create the dataset with x, y, and z as separate variables
# dataset = xr.Dataset({
#     'var': data_array,
#     'z': ('dim1', z),
#     'y': ('dim2', y),
#     'x': ('dim3', x)
# })
# dataset.to_netcdf('hey.nc')